In [1]:
import numpy as np
from sklearn.cluster import DBSCAN,KMeans
from video import videolike as vl, vid as v
from video.conversion import Conversions
import matplotlib.pyplot as plt
import ipywidgets as widgets
from PIL import Image
from IPython.display import display
import base64
import cv2 as cv
from importlib import reload

In [3]:
reload(vl)
reload(v)
resize = (100,100)
pokemon = v.Video.from_file('videos/Banned Pokemon Seizure Scene.mp4',Conversions.RGB,resize=resize)
pokevid = pokemon[300:500].blur((5,5)).vid

In [4]:
pokeflat = pokevid.reshape((-1,3))

num_pixels = pokeflat.shape[0]
num_frames = pokevid.shape[0]
width = pokevid.shape[2]
height = pokevid.shape[1]
frame_xy_positions = np.zeros((num_pixels, 3), dtype=np.int32)

frame_num, y, x = np.indices((num_frames, height, width))
frame_xy_positions[:, 0] = frame_num.ravel()
frame_xy_positions[:, 1] = y.ravel()
frame_xy_positions[:, 2] = x.ravel()
pokedata = np.concatenate([pokeflat,frame_xy_positions],axis=1)


In [16]:
dbscan = DBSCAN(eps=5,min_samples=30,n_jobs=2)
clusters = dbscan.fit_predict(pokedata)


In [14]:
clusters = clusters.reshape((pokevid.shape[0],pokevid.shape[1],pokevid.shape[2],1))
clusters.max()

1859

In [15]:
def update_frame(frame):
    plt.axis('off')
    plt.subplot(1,2,1)
    plt.imshow(pokevid[frame])
    plt.subplot(1,2,2)
    plt.imshow(clusters[frame],cmap='gray')
    plt.show()
frame_slider = widgets.IntSlider(min=0, max=pokevid.shape[0]-1, step=1, value=0,continuous_update=False)
widgets.interact(update_frame,frame=frame_slider)

interactive(children=(IntSlider(value=0, continuous_update=False, description='frame', max=199), Output()), _d…

<function __main__.update_frame(frame)>

## Trying with opencv scansegment